In [2]:
from utils.SystemMonitor import SystemMonitor
from utils.Table import Table
import os

monitor = SystemMonitor()

Might want to drop notebook for sampling because of jupyter and kernel overhead

In [4]:
file_name = "Samples1.csv"

# ONLY RUNS ON NEW SAMPLE GROUP
if not os.path.exists("dataframes/" + file_name):
    Table.create_empty_df(file_name, ["TimeStamp", "Name", "CPU_CORES", "CPU_CORES_LOG", "CPU_FREQ", "RAM_TOTAL_GB", "RAM_AV_GB", "RAM_USED_GB", "SWAP_TOTAL_GB", "SWAP_USED_GB", "CPU%", "RAM%", "RC", "WC", "RB", "WB", "dt", "PRED"])
else:
    raise(FileExistsError)

FileExistsError: 

# **ResNet50**

In [5]:
from models.SS_ResNet50 import SS_ResNet50
ss_resnet50 = SS_ResNet50()

monitor.start(interval=0.1)
ss_resnet50.predict()
monitor.stop()

# monitor.compute_and_print_average()
# print("\nPrediction correct: " + str(ss_resnet50.correct_result()))

Table.append_data_to_samples(file_name, "ResNet50", monitor, ss_resnet50)


Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 1s 999ms/step
Monitoring stopped.
1 row(s) appended to Samples1.csv.
Appended to sample file:
['23:10:27:12:52:25', 'ResNet50', 4, 4, 2808.004, '11.68', '7.94', '3.33', '0.95', '0.00', 39.333333333333336, 31.880952380952376, 0.0, 26.952380952380953, 0.0, 157598.47619047618, 0.1, 'True']
